# Relazione secondo progetto algoritmi avanzati

Francesca Meneghello 1227939 <br />
Leonardo Pratesi 1237582

## 1. Introduzione

scelta di utilizzare matrice per rappresentazione grafi

## 2. Algoritmo esatto di Held e Karp

## 3. Euristiche costruttive
### 3.1 Closest Insertion
### 3.2 Farthest Insertion

## 4. Algoritmo 2-approssimato

## 5. Tabella

| Tempi e risultati |            |                |                |                |                |
|--------------|-----------------|----------------|----------------|----------------|----------------|
|              |                 | Held e Karp    |                | Header 3       |        <       |
|              | Soluzione       | Tempo          | Errore         | Subheader 3.1  | Subheader 3.2  |
|==============|-----------------|----------------|----------------|----------------|----------------|
| burma14.tsp  | 3row, 3col span |       <        |        <       | Colspan only   |        <       |
| ulysses16.tsp|       ^         |       <        |        <       | Rowspan only   | Cell           |
| ulysses22.tsp|       ^         |       <        |        <       |       ^        | Cell           |
| eil51.tsp    |  Row            |  Each cell     |:   Centered   :| Right-aligned :|: Left-aligned  |
|.            .|. with multiple .|. has room for .|.  multi-line  .|.   multi-line .|. multi-line   .|
|.            .|. lines.        .|. more text.   .|.     text.    .|.        text. .|. text.        .|

## 6. Conclusioni